In [ ]:
# FIRST TASK
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import linear_model
from scipy import stats

# chose 5 numerical columns
chosen_cols = ['Respondent', 'Age', 'Age1stCode', 'YearsCode', 'YearsCodePro', 'CompTotal', 'WorkWeekHrs']
df = pd.read_csv("C:/Users/macko/Repos/Machine_learning/survey_results_public.csv", 
                 usecols=chosen_cols,
                 index_col='Respondent').dropna()

# checking types
print(df.dtypes, '\n')

# change string values to numbers
df.loc[df['YearsCode'] == 'Less than 1 year'] = 0
df.loc[df['YearsCode'] == 'More than 50 years'] = 60
df.loc[df['YearsCodePro'] == 'Less than 1 year'] = 0
df.loc[df['Age1stCode'] == 'Younger than 5 years'] = 4

# change types to integer
df['Age'] = df['Age'].astype(int)
df['YearsCode'] = df['YearsCode'].astype(int)
df['YearsCodePro'] = df['YearsCodePro'].astype(int)
df['Age1stCode'] = df['Age1stCode'].astype(int)
print(df.dtypes)

# sort out invalid values
df = df[(df['Age'] > int(15)) & (df['Age'] < int(90)) & (df['CompTotal'] < 5e8) & (df['WorkWeekHrs'] <= 168)]

# correlation map
sns.heatmap(df.corr(), annot=True)
plt.show()

In [ ]:
cols = df.columns.values.tolist()
i = 0

# display correlation plots
for x_val in df:
    for i in range (cols.index(x_val), 5):
#         if sec_var != st_var: # comparing same variable makes no sense
        plt.plot(df[x_val], df[cols[i + 1]], 'rp', markersize=0.5)
        plt.xlabel(x_val)
        plt.ylabel(cols[i + 1])
        plt.title('Correlation: ' + str(x_val) + ' vs ' + cols[i + 1])
        plt.show()
    
# dependent variable y --> 'Comptotal'
# independent variable x1 --> 'Age'
# independent variable x2 --> 'Age1stCode'